# Benchmarking sigopt for comparison to rocketsled optimizers

Julien Brenneck - 08/07/18

Ended up not using sigopt optimizers, could only run 10 "experiments" per month on academic tier account, statistical comparison would require a few hundred. 

In [1]:
import numpy as np
# np.random.seed(123)
from time import time

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import matplotlib.pyplot as plt
plt.style.use('seaborn')

import seaborn
seaborn.set_color_codes()

# plt.rcParams["figure.figsize"] = (10,6)

from skopt.benchmarks import branin, hart6
from skopt import gp_minimize, forest_minimize, dummy_minimize

from sigopt import Connection

In [2]:
hartdim = [(0.0, 1.0)] * 6
rosedim = [(-5.0, 10.0)] * 2
branindim = [(-5.0, 10.0), (0.0, 15.0)]

def rose(x):
    """Rosenbrock function - Extensible to N dimensions, but we using 2"""
    x = np.asarray(x)
    r = np.sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0, axis=0)
    return r

In [21]:
def eval_rose(assignments):
    return -rose([assignments['x'], assignments['y']])

def eval_bran(assignments):
    return -branin([assignments['x'], assignments['y']])

def eval_hart(assignments):
    return -hart6([assignments['x1'], assignments['x2'],assignments['x3'],assignments['x4'],assignments['x5'],assignments['x6']])

In [4]:
def plot_err(data, log=True):
    plt.figure(figsize=(12,6))
    y = np.minimum.accumulate(data, axis=1)
    mean, std = np.mean(y, axis=0), np.std(y, axis=0)
    p25, p75 = np.percentile(y, [12.5, 87.5], axis=0)
    plt.plot(mean, 'o-')
    if log:
        plt.yscale('log')
    plt.plot(p25, '--b', alpha=0.5)
    plt.plot(p75, '--b', alpha=0.5)
    plt.fill_between(np.arange(50), mean-std, mean+std, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
def plot_comp(gp_data, rf_data, rd_data, log=True, std=False, quant=False):
    y1 = np.mean(np.minimum.accumulate(gp_data, axis=1), axis=0)
    y2 = np.mean(np.minimum.accumulate(rf_data, axis=1), axis=0)
    y3 = np.mean(np.minimum.accumulate(rd_data, axis=1), axis=0)

    plt.figure(figsize=(12,6))
    plt.plot(y1, 'o-', label="GP")
    plt.plot(y2, 'v-', label="RF")
    plt.plot(y3, '^-', label="RAND")
    if log:
        plt.yscale('log')
    plt.legend()
    plt.tight_layout()

In [ ]:
conn = Connection(client_token=None)

In [6]:
experiment = conn.experiments().create(
  name="Rosenbrock",
  parameters=[
    dict(
      name="x",
      bounds=dict(
        min=-5.0,
        max=10.0
        ),
      type="double"
      ),
    dict(
      name="y",
      bounds=dict(
        min=-5.0,
        max=10.0
        ),
      type="double"
      )
    ],
  observation_budget=50
  )

print("Created experiment: https://sigopt.com/experiment/" + experiment.id)

Created experiment: https://sigopt.com/experiment/47452


In [7]:
for _ in range(experiment.observation_budget):
    suggestion = conn.experiments(experiment.id).suggestions().create()
    assignments = suggestion.assignments
    value = eval_rose(assignments)

    conn.experiments(experiment.id).observations().create(
        suggestion=suggestion.id,
        value=value
    )

In [8]:
assignments = conn.experiments(experiment.id).best_assignments().fetch().data[0].assignments

print(assignments)

Assignments({
  "x": 0.5681412021059522,
  "y": 0.24070836539448592
})


In [17]:
experiment_bran = conn.experiments().create(
  name="Branin",
  parameters=[
    dict(
      name="x",
      bounds=dict(
        min=-5.0,
        max=10.0
        ),
      type="double"
      ),
    dict(
      name="y",
      bounds=dict(
        min=0.0,
        max=15.0
        ),
      type="double"
      )
    ],
  observation_budget=50
  )

print("Created experiment: https://sigopt.com/experiment/" + experiment.id)

Created experiment: https://sigopt.com/experiment/47452


In [18]:
for _ in range(experiment_bran.observation_budget):
    suggestion = conn.experiments(experiment_bran.id).suggestions().create()
    assignments = suggestion.assignments
    value = eval_bran(assignments)

    conn.experiments(experiment_bran.id).observations().create(
        suggestion=suggestion.id,
        value=value
    )